<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 5
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec.
1. Classification supervisée (cours 9) utilisant des traits lexicaux :
  1. les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
  1. apparition de mots indicateurs dans le voisinage de *interest*.

Les méthodes (1) et (2) n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La méthode (3) vise à classifier les occurrences de *interest*, les sens étant les classes, en utilisant comme traits les mots du contexte (ce sera de l'apprentissage supervisé).

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez brièvement répondre aux questions suivantes :

1. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Est-ce qu'om considère le pluriel *interests* aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences du mot ?

In [ ]:
# 1. Le fichier zip: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz et le fichier README: https://www.d.umn.edu/~tpederse/Data/README.int.txt
# 2. Fichier texte avec des phrases séparée par des $$ et les mots interest/s avec un underscore et un id pour désigner leur sens. Le pluriel est considéré. Une seule occurence est traitée par phrase. Si la phrase comporte plusieurs occurence, la phrase sera copié et la prochaine occurence sera traîtée. Les occurences non-traitée sont précédées d'une *

3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [ ]:
# Sense 1 =  361 occurrences (15%) - readiness to give attention
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# Sense 5 =  500 occurrences (21%) - a share in a company or business
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [ ]:
# Le dictionnaire est LDOCE (Longman Dictionary of Contemporary English)
# Sense 1 = "something you want to know or learn more about"
# Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sense 4 = "the things that bring advantages to someone or something"
# Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sense 6 = "the extra money that you must pay back when you borrow money"

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [ ]:
# Veuillez répondre ici (en commentaire) à la question.
# Il y a 7 synsets.
# Sense 1 = "a sense of concern with and curiosity about someone or something"
# Sense 2 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
# Sense 3 = "a diversion that occupies one's time and thoughts (usually pleasantly)"
# Sense 4 = "a reason for wanting something done"
# Sense 5 = "a right or legal share of something; a financial involvement with something"
# Sense 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"

6. Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [ ]:
import nltk

In [ ]:
from random import randrange

In [ ]:
readme_definitions = [
"readiness to give attention",
"quality of causing attention to be given to",
"activity, etc. that one gives attention to",
"advantage, advancement or favor",
"a share in a company or business",
"money paid for the use of money"
]
dictionnary_definitions = [
"if you have an interest in something or someone, you want to know or learn more about them",
"an activity that you enjoy doing or a subject that you enjoy studying",
"a quality or feature of something that attracts your attention or makes you want to know more about it",
"the extra money that you must pay back when you borrow money",
"the things that bring advantages to someone or something",
"if you have an interest in a particular company or industry, you own shares in it"
]
synSet_definitions = []

In [ ]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).

senses1 = [[word for word in definition.split() if len(word) > 3] for definition in readme_definitions]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity,', 'etc.', 'that', 'gives', 'attention'], ['advantage,', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'money']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [ ]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = np.array(["readiness to give attention something you want to know or learn more about",
           "quality of causing attention to be given to a quality or feature of something that attracts your attention or makes you want to know more about it",
           "activity, etc. that one gives attention to an activity that you enjoy doing or a subject that you enjoy studying",
           "advantage, advancement or favor the things that bring advantages to someone or something",
           "a share in a company or business if you have an interest in a particular company or industry, you own shares in it",
           "money paid for the use of money the extra money that you must pay back when you borrow money"])

senses2 = [[word for word in np.unique(sentence.split()) if len(word) > 4] for sentence in senses2]


print(senses2)

8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [ ]:
# Veuillez répondre ici à la question.
import re
sentences = []
with open("interest-original.txt", 'r') as f:
  current_line = ""
  for line in f.readlines():
      if line == "$$\n":
        sentences.append(re.sub("  ", " ", current_line).split())
        current_line = ""
      else:
        current_line += re.sub(r"[^\w ]*", "", line)


print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', 'has', 'nt', 'stalled', 'at', 'all', 'mr', 'hines', 'maintains'], ['it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr', 'gorbachev', 'succeed'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', 'expression', 'of', 'interest_1', 'because', 'under', 'chapter', '11', 'revco', 'has', 'exclusivity', 'rights', 'until', 'feb', '28']]


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [ ]:
# Veuillez répondre ici à la question.


Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [ ]:
# Veuillez répondre ici à la question.


En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [ ]:
# Veuillez répondre ici à la question.


## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [ ]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "../../../gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [ ]:
# Veuillez répondre ici à la question.


Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec les précédents ?

In [ ]:
# Veuillez répondre ici à la question.


## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

### 3.A. Traits lexicaux positionnels

Dans cette première représentation des traits, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python suivi de l'index du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
L'index du sens servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [30]:
# Veuillez répondre ici à la question.
def get_index_of_regexp_match(regexp, words):
  for index, value in enumerate(words):
    if re.match(regexp, value):
      return index
  return -1

def create_dict_around(sentence, index):
  end = len(sentence)
  [word,sense] = sentence[index].split("_")
  neighbours = {}
  neighbours['word-2'] = sentence[index - 2] if index >= 2  else 'NONE'
  neighbours['word-1'] = sentence[index - 1] if index >= 1  else 'NONE'
  neighbours['word0']  =  word
  neighbours['word+1'] = sentence[index + 1] if index < end - 1  else 'NONE'
  neighbours['word+2'] = sentence[index + 2] if index < end - 2  else 'NONE'
  return [neighbours , sense]

def extract_feature_A(sentence):
  index = get_index_of_regexp_match(r"^.*_\d$", sentence)
  return create_dict_around(sentence, index)

items_with_features_A = [extract_feature_A(sent) for sent in sentences]
print(len(items_with_features_A))
print(items_with_features_A[151:154])

2368
[[{'word-2': 'NONE', 'word-1': 'investor', 'word0': 'interest', 'word+1': 'in', 'word+2': 'stock'}, '1'], [{'word-2': 'the', 'word-1': 'western', 'word0': 'interest', 'word+1': 'to', 'word+2': 'see'}, '4'], [{'word-2': 'expression', 'word-1': 'of', 'word0': 'interest', 'word+1': 'because', 'word+2': 'under'}, '1']]


On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [33]:
from random import shuffle
import math

In [43]:
iwf_A_train = []
iwf_A_test  = []
# Veuillez répondre ici à la question.
def separate_list(the_list, train_rate= 0.8):
  index = math.floor(len(the_list) * train_rate)
  shuffle(the_list)
  return (the_list[:index], the_list[index:])

print(separate_list([0,1,2,3,4,5,6,7,8,9,10]))

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

([1, 8, 4, 3, 10, 9, 0, 7], [6, 5, 2])
0   0
[] []


Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs (voir la doc NLTK), et pouvez-vous expliquer cet affichage ?

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


In [ ]:
# Question supprimée


### 3.B. Présence de mots indicateurs

Une deuxième façon d'encoder les traits lexicaux est de constituer un vocabulaire avec les mots qui apparaissent dans le voisinage de *interest* et de définir ces mots comme traits.  Par conséquent, pour chaque occurrence de *interest*, vous allez extraire la valeur de ces traits sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'* sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui est décrite, et le dernier nombre est le sens, entre 1 et 6.

Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [ ]:
word_list = []
# Veuillez répondre ici à la question.

print(len(word_list))
print(word_list[:50])

En utilisant un objet `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi faire varier ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [ ]:
# Veuillez répondre ici à la question.

print(vocabulary[:50])

Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 3B.  Cet ensemble sera appelé `items_with_features_B`.

In [ ]:
items_with_features_B = []
# Veuillez répondre ici à la question.

print(len(items_with_features_B))

Comme dans la section 3A, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [ ]:
iwf_B_train = []
iwf_B_test  = []
# Veuillez répondre ici à la question.

print(len(iwf_B_train), ' ', len(iwf_B_test))

Comme pour la section 3A, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_train`.  Veuillez obtenir le score de ce classifieur.  En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs ?

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


In [ ]:
# Veuillez recopier ici en conclusion les scores des quatre 
# expériences, pour pouvoir les comparer d'un coup d'oeil.


## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.